NLP on words without stemming

In [1]:
# Import Packages
import pandas as pd
from nltk import word_tokenize
from gensim import models, corpora
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
import re
from gensim import similarities
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
import pyLDAvis.sklearn
output_notebook()

Loading BokehJS ...

In [2]:
# Import the cleaned data
data = pd.read_csv("cleaned_data.csv")

In [3]:
# Remove first column
data =data.iloc[:,1:3]

In [4]:
# Let's look at the shape of the dataset
data.shape

(3890, 2)

In [5]:
# Tokenized the words for modeling
data['sentence'] = data['sentence'].apply(lambda x: word_tokenize(x) )

### LDA Model 
-> for Topic Modeling

In [6]:
# Create dictionary on the tokenzied data
dictionary = corpora.Dictionary(data['sentence'])

# Create corpus on the data
corpus = [dictionary.doc2bow(w) for w in data['sentence']]

# Set number of topics to 15
topics = 15

# Create LDA model
lda= models.LdaModel(corpus=corpus, num_topics=topics, id2word=dictionary)

In [7]:
for i in range(topics):
    print("[Topic %s] :" % i, lda.print_topic(i, 15))

[Topic 0] : 0.011*"act" + 0.009*"evidence" + 0.008*"court" + 0.008*"would" + 0.008*"said" + 0.007*"notice" + 0.007*"made" + 0.006*"may" + 0.006*"applicant" + 0.005*"also" + 0.005*"case" + 0.005*"order" + 0.005*"time" + 0.005*"first" + 0.005*"claim"
[Topic 1] : 0.014*"court" + 0.012*"application" + 0.010*"claim" + 0.010*"would" + 0.009*"made" + 0.009*"act" + 0.007*"may" + 0.006*"applicant" + 0.005*"said" + 0.005*"appeal" + 0.005*"whether" + 0.005*"scheme" + 0.005*"evidence" + 0.005*"case" + 0.005*"order"
[Topic 2] : 0.038*"tribunal" + 0.020*"appellant" + 0.013*"decision" + 0.012*"applicant" + 0.010*"evidence" + 0.009*"application" + 0.008*"court" + 0.008*"would" + 0.008*"appeal" + 0.007*"made" + 0.007*"act" + 0.007*"review" + 0.007*"information" + 0.007*"hearing" + 0.007*"federal"
[Topic 3] : 0.014*"court" + 0.013*"act" + 0.011*"would" + 0.008*"made" + 0.008*"may" + 0.007*"order" + 0.007*"respondent" + 0.006*"application" + 0.006*"evidence" + 0.005*"first" + 0.005*"claim" + 0.005*"case"

In [18]:
# Let's try some sentence from the document and see which topic it belongs to
def clean(text):
    stop = stopwords.words('english')
    text = text.lower()
    text = word_tokenize(text)
    return text

In [19]:
# Got this text from 06_13.xml which has name: Skymaker Holdings Pty Ltd v Jadjet Pty Ltd [2006] FCA 13 (20 January 2006)
text = """The Brochure represented that the child care business had a weekly turnover of $7250,
was strong and profitable in trading and had the potential to expand.In particular,
I held that it was open to the respondents to submit that the Commonwealth was precluded from recovering its loss for the reasons outlined in their letters to the applicants and to contest the recoverability and quantification of any loss specified by the applicants."""

In [20]:
text = re.sub(r'[^\w\s]','',text)

In [25]:
text

'The Brochure represented that the child care business had a weekly turnover of 7250\nwas strong and profitable in trading and had the potential to expandIn particular\nI held that it was open to the respondents to submit that the Commonwealth was precluded from recovering its loss for the reasons outlined in their letters to the applicants and to contest the recoverability and quantification of any loss specified by the applicants'

In [21]:
bow = dictionary.doc2bow(clean(text)) 
print(lda[bow]) 

[(5, 0.3610212), (6, 0.5977088)]


This means this paragrah belongs to the topic 11 with probability of 95.56%
Let's see the similar document with this one and give the names of the most 5 similar documents

In [22]:
# Get the similar topics 
lda_index = similarities.MatrixSimilarity(lda[corpus])

similarities = lda_index[lda[bow]]

# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda i: -i[1])

In [23]:
# Let's see what's the most 5 similar documents from the original documents
for i in similarities[:5]:
    print(data.iloc[i[0],0])
    print("=="*30)

E amp J Gallo Winery v Lion Nathan Australia Pty Limited 2008 FCA 934 20 June 2008
Television Food Network GP v Food Channel Network Pty Ltd No 2 2009 FCA 271 27 March 2009
Global Brand Marketing Inc v YD Pty Ltd 2008 FCA 605 7 May 2008
Colorado Group Limited v Strandbags Group Pty Limited No 2 2006 FCA 880 7 July 2006
Bing Software Pty Ltd v Bing Technologies Pty Limited No 1 2008 FCA 1760 25 November 2008


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_df = 0.9, min_df = 0.01, stop_words = 'english', lowercase = True,
                      token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
vectdata= vect.fit_transform(clean_data['sentence'])

In [54]:
trunc= TruncatedSVD(n_components=2)
t_data = vectdata.T
trunc_word = trunc.fit_transform(t_data)
 
temp = pd.DataFrame(columns=['x', 'y', 'text'])

temp['text'] = vect.get_feature_names()
temp['x'] = trunc_word[:,0]
temp['y'] = trunc_word[:, 1]
source = ColumnDataSource(ColumnDataSource.from_df(temp))

plt = figure(plot_width= 600, plot_height=550)
plt.circle("x", "y", size=10, source=source, fill_alpha=0.5)
label = LabelSet(x = 'x', y = 'y', text = 'text', 
                 text_font_size = '10pt', text_align = 'center', 
                 source = source, y_offset = 5)
plt.add_layout(label)
show(plt, notebook_handle=True)

pyLDAvis.sklearn package is good for visualizing the Topic Model. Let's use it to show

Because this package need the LDA model created by sklearn. So I need to create new one now

In [86]:
df = pd.read_csv("cleaned_data.csv")
df.head(2)

,Unnamed: 0,name,sentence
0,0,University of Western Australia v Gray No 21 2...,application brought university western gray me...
1,1,Parker v Parker 2009 FCA 930 25 August 2009,introduction applicant review decision adminis...


In [89]:
df = pd.read_csv("cleaned_data.csv")
vect = CountVectorizer(min_df=0.01, max_df=0.9)
vect_data = vect.fit_transform(df.iloc[:,2])
 
# Build LDA model
n_topics= 15
lda= LatentDirichletAllocation(n_components=n_topics, max_iter=10, learning_method='online')
lda_Z = lda.fit_transform(vect_data)

In [90]:
# Dash board with relative words is showed below
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, vect_data, vect, mds='tsne')

panel

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
10    -153.508423  -41.773277       1        1  11.553105
9      302.647125   39.576981       2        1   9.935290
2      137.752899 -287.108643       3        1   9.672731
11     127.951065  -76.006180       4        1   9.220820
13     229.855667  234.775818       5        1   8.857873
6      -51.964485 -352.842865       6        1   6.333481
8     -289.661835  113.609871       7        1   5.904755
1      -23.434801 -150.194611       8        1   5.743186
14    -216.617493 -238.906784       9        1   5.630814
12    -349.341705  -83.617638      10        1   5.343086
7      302.634094 -168.800217      11        1   4.909023
4       30.222757  292.765106      12        1   4.669366
5      -72.761391  106.668503      13        1   4.516189
0       94.954407   90.273048      14        1   4.057369
3     -174.609161  278.389404      15        1   3.652910, topic_info=     Category          Freq            Term         Total  loglift  logprob
term                                                                       
4769  Default  56996.000000        tribunal  56996.000000  30.0000  30.0000
263   Default  28260.000000       appellant  28260.000000  29.0000  29.0000
163   Default  28084.000000       agreement  28084.000000  28.0000  28.0000
268   Default  61976.000000       applicant  61976.000000  27.0000  27.0000
3963  Default  38600.000000      respondent  38600.000000  26.0000  26.0000
257   Default  23376.000000          appeal  23376.000000  25.0000  25.0000
1177  Default  40276.000000        decision  40276.000000  24.0000  24.0000
3124  Default  25089.000000          notice  25089.000000  23.0000  23.0000
4729  Default  11317.000000           trade  11317.000000  22.0000  22.0000
3222  Default  35177.000000           order  35177.000000  21.0000  21.0000
4094  Default  14265.000000          scheme  14265.000000  20.0000  20.0000
4695  Default   8903.000000           title   8903.000000  19.0000  19.0000
2643  Default   8925.000000            land   8925.000000  18.0000  18.0000
1841  Default  21776.000000         federal  21776.000000  17.0000  17.0000
3048  Default   7505.000000          native   7505.000000  16.0000  16.0000
805   Default  22307.000000         company  22307.000000  15.0000  15.0000
5033  Default   9907.000000            visa   9907.000000  14.0000  14.0000
2955  Default  17111.000000        minister  17111.000000  13.0000  13.0000
706   Default  36975.000000           claim  36975.000000  12.0000  12.0000
2865  Default   6583.000000            mark   6583.000000  11.0000  11.0000
2350  Default   9436.000000          income   9436.000000  10.0000  10.0000
3317  Default   6105.000000          patent   6105.000000   9.0000   9.0000
4579  Default   9219.000000             tax   9219.000000   8.0000   8.0000
297   Default   8766.000000            area   8766.000000   7.0000   7.0000
5172  Default  13826.000000            work  13826.000000   6.0000   6.0000
2076  Default   6716.000000           goods   6716.000000   5.0000   5.0000
2911  Default  12484.000000         meeting  12484.000000   4.0000   4.0000
1565  Default   7685.000000      employment   7685.000000   3.0000   3.0000
1674  Default  59372.000000        evidence  59372.000000   2.0000   2.0000
2824  Default   8844.000000      magistrate   8844.000000   1.0000   1.0000
...       ...           ...             ...           ...      ...      ...
535   Topic15    859.875576           brand   1401.911148   2.8208  -5.7118
2855  Topic15    356.073634    manufacturer    516.770578   2.9372  -6.5934
915   Topic15    511.867663       confusion    800.962611   2.8619  -6.2305
4277  Topic15   1667.444886            sold   3397.416626   2.5979  -5.0495
3037  Topic15   2272.017202            name   5735.591802   2.3836  -4.7401
4956  Topic15   4142.722398             use  13346.518495   2.1397  -4.1394
972   T